In [3]:
import spacy
nlp = spacy.load('en_core_web_lg')
EF_sent = [i.replace('\n','')for i in open('EF666.txt').readlines()]

In [4]:
import spacy
from spacy.tokens import Doc
class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(' ')
        # All tokens 'own' a subsequent space character in this tokenizer
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)
nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

In [5]:
## get have VT_tag sentence
Tmp_VT = []
for i in EF_sent:
    if 'VT+' in i or 'VT-' in i:
        Tmp_VT.append(i)

In [19]:
Tmp_VT[10]

'When I arrived there , I saw that someone {+had//VT+} forced the lock of the door .'

In [6]:
def diff2before_after_taglook(text , tag):
    before, after , tmp_V = [], [] ,[]
    for token in text.split(' '):
        if token.startswith('{+'):
            after_word = token[2:-2].split('//')[0].replace('\u3000',' ')
            t = token[2:-2].split('//')[1]
            if t==tag:
                after.append(token)
                tmp_V.append(token)
            else:
                after.append(after_word)
        elif token.startswith('[-'):
            if token.endswith('+}'):
                delete_tmp, insert_tmp = token[2:-2].split('-]{+')
                delete = delete_tmp.split('//')[0].replace('\u3000',' ')
                insert = insert_tmp.split('//')[0].replace('\u3000',' ')
                t = insert_tmp.split('//')[1]
                if t==tag:
                    before.append(token)
                    after.append(token)
                    tmp_V.append((token))
                else:
                    before.append(delete)
                    after.append(insert)
            else:
                before_word = token[2:-2].split('//')[0].replace('\u3000',' ')
                t = token[2:-2].split('//')[1]
                if t==tag:
                    before.append(token)
                    tmp_V.append(token)
                else:
                    before.append(before_word)
        else:
            toke = token.replace('\u3000',' ')
            before.append(token)
            after.append(token)
    return  ' '.join(after) , tmp_V
def diff_after_before(text):
    before, after = [], []
    for token in text.split(' '):
        if token.startswith('{+'):
            after_word = token[2:-2].split('//')[0]
            after.append(after_word)
        elif token.startswith('[-'):
            if token.endswith('+}'):
                delete_tmp, insert_tmp = token[2:-2].split('-]{+')
                delete = delete_tmp.split('//')[0]
                insert = insert_tmp.split('//')[0]
                before.append(delete)
                after.append(insert)
            else:
                before_word = token[2:-2].split('//')[0]
                before.append(before_word)
        else:
            before.append(token)
            after.append(token)
    return ' '.join(before).replace('\u3000',' '), ' '.join(after).replace('\u3000',' ')

In [7]:
def find_seg_index(token):
    l = len(token)
    seg_index_tmp=[]
    for index, word in enumerate(token):
        if 'VT' in word:
            seg_index_tmp.append((index-1,index,index+1))
    seg_ngram_index = []
    for ii in seg_index_tmp:
        seg_index = []
        for i in ii:
            if i<0:
                seg_index.append(0)
            elif i>=l:
                seg_index.append(l-1)
            else:
                seg_index.append(i)
        seg_index = sorted(set(seg_index))
        seg_ngram_index.append(seg_index)
    return seg_ngram_index

In [8]:
def find_segment(tag_sen , tag_arry):
    token = tag_sen.split(' ')
    index_arry = find_seg_index(token)
    segment_arry = []
    for index , tag in zip(index_arry , tag_arry):
        seg = ' '.join([token[i] for i in index])
        before , after = diff_after_before(seg)
        segment_arry.append((before , after))
    return segment_arry

In [9]:
def find_segment_position(full, part):
    full = full.split()
    part = part.split()
    length = len(part)
    for i, sublist in enumerate((full[i:i+length] for i in range(len(full)))):
        if part == sublist:
            return [i+c for c in range(length)]
    return []

In [10]:
def spacy_index_match(sen , index):
    all_tag = ['VBP' , 'VBZ' , 'VBG' , 'VBN' , 'VBD' , 'VB']
    word = ['will']
    doc = nlp(sen)
    show = []
    for i in index:
        if doc[i].text in word or doc[i].tag_ in all_tag:
            show.append((doc[i].text , doc[i].tag_))
    return show

In [11]:
def find_pattern_spacy_result(sen ,pattern='VT'):
    tag_sen , tag = diff2before_after_taglook(sen , pattern)
    before , after = diff_after_before(tag_sen)
    before  = ' '.join([i for i in before.split(' ') if i])
    after = ' '.join([i for i in after.split(' ') if i])
    segment_arry = find_segment(tag_sen , tag)
    result = []
    for segment in segment_arry:
        before_seg = segment[0]
        after_seg = segment[1]
        b_index = find_segment_position(before , before_seg)
        a_index = find_segment_position(after , after_seg)
        b_spacy_result = spacy_index_match( before , b_index)
        a_spacy_result = spacy_index_match( after , a_index)
        result.append((b_spacy_result , a_spacy_result))
        #print(b_spacy_result , a_spacy_result)
    return result , tag_sen

In [12]:
def now(pair):
    try:
        if not future(pair):
            if pair[0][1]=='VBP' or pair[0][1]=='VBZ':
                return 'now'
    except:pass
def past(pair):
    try:
        if not future(pair):
            if pair[0][1]=='VBD':
                return 'past'
    except:pass
def future(pair):
    try:
        if ((pair[0][0]=='is' or pair[0][0]=='are' or pair[0][0]=='am')  and pair[1][0]=='going') or pair[0][0]=='will':
            if pair[-1][1]=='VB':
                return 'future'
    except:pass

In [13]:
def simple(pair):
    try:
        if not nowing(pair) and not finish(pair) and not finishing(pair):
            if now(pair) or past(pair) or future(pair):
                return 'simple'
    except:pass
def nowing(pair):
    try:
        if len(pair)==2:
            if pair[0][0]=='is' or pair[0][0]=='are' or pair[0][0]=='am' or pair[0][0]=='was' or pair[0][0]=='were':
                if pair[-1][1]=='VBG':
                    return 'nowing'
            if pair[0][0]=='will':
                if pair[1][0]=='be':
                    if pair[-1][1]=='VBG':
                        return 'nowing'
    except:pass
def finish(pair):
    try:
        if len(pair)==2:
            if pair[0][0]=='have' or pair[0][0]=='has'or pair[0][0]=='had':
                if pair[-1][1]=='VBN' and pair[-1][0] != 'been':
                    return 'finish'
        if len(pair)==3:
            if pair[0][0]=='will':
                if pair[1][0]=='have':
                    if pair[-1][1]=='VBN' and pair[-1][0] != 'been':
                        return 'finish'
    except:pass
def finishing(pair):
    try:
        if len(pair)==3:
            if pair[0][0]=='have' or pair[0][0]=='has'or pair[0][0]=='had':
                if pair[1][0]=='been':
                    if pair[-1][1]=='VBG':
                        return 'finishing'
        if len(pair)==4:
            if pair[0][0]=='will':
                if pair[1][0]=='have':
                    if pair[2][0]=='been':
                        if pair[-1][1]=='VBG':
                            return 'finishing'
    except:pass

In [14]:
def find_time(tmp , now , past , future):
    if not tmp:
        if now : tmp = now
        elif past : tmp = past
        elif future : tmp = future
    return tmp
def find_tense(tmp , simple , nowing , finish , finishing):
    if not tmp:
        if simple : tmp = simple
        elif nowing : tmp = nowing
        elif finish : tmp= finish
        elif finishing : tmp = finishing
    return tmp

In [15]:
def match(spacy_result , tag_sen):
    Rule = []
    for pair in spacy_result:
        if pair[0] and pair[1]:
            before , after = pair[0] , pair[1]
            tmp_b , tmp_bb , tmp_a , tmp_aa  = '' , '' , '' , ''
            now_b , past_b , future_b = now(before) , past(before) , future(before)
            now_a , past_a , future_a = now(after) , past(after) , future(after)
            #print(now_b , past_b , future_b)
            #print(now_a , past_a , future_a)
            simple_b , nowing_b , finish_b , finishing_b = simple(before) , nowing(before) , finish(before) , finishing(before)
            simple_a , nowing_a , finish_a , finishing_a = simple(after) , nowing(after) , finish(after) , finishing(after)
            tmp_b = find_time(tmp_b , now_b , past_b , future_b)
            tmp_bb = find_tense(tmp_bb , simple_b , nowing_b , finish_b , finishing_b )
            tmp_a = find_time(tmp_a , now_a , past_a , future_a)
            tmp_aa = find_tense(tmp_aa , simple_a , nowing_a , finish_a , finishing_a )
            #print(tmp_b , tmp_a)
            #print(tmp_bb , tmp_aa)
            if tmp_a and tmp_b:
                rule1 = tmp_b + '->' + tmp_a
                Rule.append((rule1 , pair))
                #print(rule)
                #print(before , after)
                #print(tag_sen)
            if tmp_aa and tmp_bb:
                rule2 = tmp_bb + '->' + tmp_aa
                Rule.append((rule2 , pair))
                #print(before , after)
                #print(tag_sen)
    return Rule

In [16]:
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
def pair_key(pair):
    key_verb =''
    before_word = [lemmatizer(i[0] , 'VERB')[0] for i in pair[0]]
    after_word = [lemmatizer(i[0] , 'VERB')[0] for i in pair[1]]
    #print(before_word , after_word)
    #list3 = set(before_word)&set(after_word)
    #list4 = sorted(list3, key = lambda k : before_word.index(k))
    return after_word[-1] 

In [17]:
from collections import defaultdict
d = defaultdict(lambda: defaultdict(lambda: []))
for index , sentence  in enumerate(Tmp_VT[:]):
    try:
        result , tag_sen = find_pattern_spacy_result(sentence)
        for item in match(result , tag_sen):
            rule , pair = item[0] , item[1]
            key = pair_key(pair)
            d[rule][key].append(index)
    except:print(index)

In [18]:
def look_index_result(index):
    result , tag_sen = find_pattern_spacy_result(Tmp_VT[index])
    for item in match(result , tag_sen):
        rule , pair = item[0] , item[1]
        key = pair_key(pair)
        print('Rule : ',rule ,"|", 'Main verb : ',key ,"|", pair )  
    print("# Sentence : #")
    print('##################################################')
    print(tag_sen)
    print('##################################################')

In [19]:
# 'simple->nowing'
# 'now->now'
# 'simple->simple'
# 'now->future'
# 'future->now'
# 'finish->finish'
# 'nowing->nowing'
# 'future->future'
# 'simple->finish'
# 'finishing->finish'
# 'nowing->simple'
# 'simple->finishing'
# 'nowing->finishing'
# 'finishing->finishing'
# 'finish->finishing'
# 'finish->simple'
# 'nowing->finish'
# 'finishing->simple'
# 'finishing->nowing'
# 'finish->nowing'

In [20]:
def look_all_rule_count(d):
    def look_rule_count(d , rule):
        count = 0
        for verb in d[rule].keys():
            count+=len(d[rule][verb])
        return count
    for rule in d.keys():
        print(rule , look_rule_count(d , rule))
    return

In [24]:
def look_rule_verb_count(d , rule , number=None):
    dd = {}
    for verb in d[rule].keys():
        count = 0
        count+=len(d[rule][verb])
        dd[verb] = count
    print(sorted(dd.items() , key = lambda x:x[1] , reverse=True)[:number])
    return 

In [25]:
#look_all_rule_count(d)

In [26]:
look_rule_verb_count(d , 'past->now' , 10)

[('be', 2501), ('have', 1081), ('work', 501), ('come', 305), ('get', 278), ('live', 273), ('like', 255), ('make', 250), ('do', 246), ('love', 224)]


In [27]:
def confuse_look(d , rule , verb , number=10):
    print("# Rule #")
    for i in d[rule][verb][:number]:
        look_index_result(i)
        print('-------------------------END--------------------------------------------')
    print('')
    print(' ')

In [28]:
confuse_look(d , 'past->now' , 'be' , 10)

# Rule #
Rule :  past->now | Main verb :  be | ([('were', 'VBD')], [('are', 'VBP')])
Rule :  simple->simple | Main verb :  be | ([('were', 'VBD')], [('are', 'VBP')])
# Sentence : #
##################################################
Plot : They [-were//VT-]{+are//VT+} in the city when vampires attack " The Twilight " is a book that captures the romance of a couple , a young vampire and a mortal girl , who struggle to get all this love to live .
##################################################
-------------------------END--------------------------------------------
Rule :  past->now | Main verb :  be | ([('was', 'VBD')], [('is', 'VBZ')])
Rule :  simple->simple | Main verb :  be | ([('was', 'VBD')], [('is', 'VBZ')])
# Sentence : #
##################################################
My name is Elaine , I 'm twenty - five years old , I 'm from Taiwan but I live in China , Guangzhou . There are seven people in my family : my father , my mother , my brother , my husband and I have three daug

In [591]:
#import dill as pickle
#with open('model.pkl', 'wb') as file:
#    pickle.dump(d, file)

In [592]:
#with open('model.pkl', 'rb') as in_strm:
#    datastruct = dill.load(in_strm)